In [41]:
# Data visualization
import matplotlib.pyplot as plt 

# Data manipulation
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile

In [42]:
# Webscraping
import glob
import requests
from bs4 import BeautifulSoup
import time
import datetime
from pandas.core.common import flatten
import os
from itertools import chain
from tqdm import tqdm
import json
import urllib.request

In [43]:
os.getcwd()

'd:\\Desktop\\Thesis\\predicting-merger-decision-outcomes\\src\\python\\notebook'

In [44]:
import sys
sys.path.append(f'../../src/python')
from scraper import createFolder, get_merger_links, download_pdf, get_merger_info

## Get merger case links

In [45]:
files = os.path.join(f"../../data/raw/", "*.txt")
file_list = glob.glob(files)

txt_list = [os.path.basename(file_path) for file_path in file_list]

In [46]:
txt_list

['art6.0_1.txt',
 'art6.0_10.txt',
 'art6.0_100.txt',
 'art6.0_101.txt',
 'art6.0_102.txt',
 'art6.0_103.txt',
 'art6.0_104.txt',
 'art6.0_105.txt',
 'art6.0_106.txt',
 'art6.0_107.txt',
 'art6.0_108.txt',
 'art6.0_109.txt',
 'art6.0_11.txt',
 'art6.0_110.txt',
 'art6.0_111.txt',
 'art6.0_112.txt',
 'art6.0_113.txt',
 'art6.0_114.txt',
 'art6.0_115.txt',
 'art6.0_12.txt',
 'art6.0_13.txt',
 'art6.0_14.txt',
 'art6.0_15.txt',
 'art6.0_16.txt',
 'art6.0_17.txt',
 'art6.0_18.txt',
 'art6.0_19.txt',
 'art6.0_2.txt',
 'art6.0_20.txt',
 'art6.0_21.txt',
 'art6.0_22.txt',
 'art6.0_23.txt',
 'art6.0_24.txt',
 'art6.0_25.txt',
 'art6.0_26.txt',
 'art6.0_27.txt',
 'art6.0_28.txt',
 'art6.0_29.txt',
 'art6.0_3.txt',
 'art6.0_30.txt',
 'art6.0_31.txt',
 'art6.0_32.txt',
 'art6.0_33.txt',
 'art6.0_34.txt',
 'art6.0_35.txt',
 'art6.0_36.txt',
 'art6.0_37.txt',
 'art6.0_38.txt',
 'art6.0_39.txt',
 'art6.0_4.txt',
 'art6.0_40.txt',
 'art6.0_41.txt',
 'art6.0_42.txt',
 'art6.0_43.txt',
 'art6.0_44.txt'

In [47]:
merger_details_full = []
for txt in txt_list:
    article = txt[:6]
    merger_details_list = get_merger_links(decision_type = txt)
    merger_details_full.append([article, merger_details_list])

In [48]:
merger_details_df = pd.DataFrame(merger_details_full, columns = ['article', 'link'])

In [49]:
# split list to rows
merger_details_df = merger_details_df.explode('link')
merger_details_df = merger_details_df.reset_index(drop=True)

In [50]:
# split list to columns
merger_details_df[['case_num', 'link']] = pd.DataFrame(merger_details_df['link'].to_list(), index=merger_details_df.index)

In [51]:
merger_details_df['article'].value_counts()

art6.0    5734
art6.1     240
art9.3     129
art8.2      73
art8.1      36
art8.3      13
Name: article, dtype: int64

In [21]:
# drop duplicates
# merger_details_df = merger_details_df.drop_duplicates(subset='link', keep='first')

In [52]:
merger_details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6225 entries, 0 to 6224
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   article   6225 non-null   object
 1   link      6225 non-null   object
 2   case_num  6225 non-null   object
dtypes: object(3)
memory usage: 146.0+ KB


In [23]:
merger_details_df

,article,link,case_num
0,art6.0,case_details.cfm?proc_code=2_M_10000,M.10000
1,art6.0,case_details.cfm?proc_code=2_M_10001,M.10001
2,art6.0,case_details.cfm?proc_code=2_M_10002,M.10002
3,art6.0,case_details.cfm?proc_code=2_M_10003,M.10003
4,art6.0,case_details.cfm?proc_code=2_M_10004,M.10004
...,...,...,...
6244,art9.3,case_details.cfm?proc_code=2_M_9421,M.9421
6245,art9.3,case_details.cfm?proc_code=2_M_9545,M.9545
6246,art9.3,case_details.cfm?proc_code=2_M_9547,M.9547
6249,art9.3,case_details.cfm?proc_code=2_M_9871,M.9871


In [53]:
merger_details_df['article'].value_counts()

art6.0    5734
art6.1     240
art9.3     129
art8.2      73
art8.1      36
art8.3      13
Name: article, dtype: int64

In [54]:
# save merger details df
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../data/scraped/merger_details_{date}.csv"
if os.path.exists(file_name):
    os.remove(file_name)

# export data
merger_details_df.to_csv(file_name, index=False)